In [ ]:
import torch
import torch.onnx
import commons

from load_checkpoint import load_checkpoint
from hparams import get_hparams_from_file
from models import SynthesizerInf
from text.symbols import symbols
from text.cleaners import japanese_phrase_cleaners
from text import cleaned_text_to_sequence

In [ ]:
hps = get_hparams_from_file("./configs/ljs_base.json")

net_g = SynthesizerInf(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

_ = load_checkpoint("./logs/ljs_base/G_119000.pth", net_g, None)

In [ ]:
torch.set_grad_enabled(False)
longtext = "どうしてこうなるんだろう。初めて、好きな人が出来た。一生ものの友だちができた。嬉しいことが二つ重なって。その二つの嬉しさが、また、たくさんの嬉しさを連れてきてくれて。夢のように幸せな時間を手に入れたはずなのに…なのに、どうして、こうなっちゃうんだろう。　どうしてこうなるんだろう。初めて、好きな人が出来た。一生ものの友だちができた。嬉しいことが二つ重なって。その二つの嬉しさが、また、たくさんの嬉しさを連れてきてくれて。夢のように幸せな時間を手に入れたはずなのに…なのに、どうして、こうなっちゃうんだろう。 えっとね……。管理人の司令官でしょ？　あとは……　レッドバロン、夜の貴公子読むだけでも恥ずかしいな。　それにエデンの女王と、ハリーって人たちが常連。　えっとね……。管理人の司令官でしょ？　あとは……　レッドバロン、夜の貴公子読むだけでも恥ずかしいな。　それにエデンの女王と、ハリーって人たちが常連。　このエデンの女王？　もさ、なんか怪しいって　思っただけで、みゃーこ先輩みたいにアニメには　ないこと書いてるってわかんなかったし。　ちゅっ、ちゅっ、じゅるるっ、全部、んぁ、んっ、　口に、んんっ、全部、飲む、からぁっ、　お兄ちゃんの、ちょうだい、ちょうだぁい……っ。　んくっ、んっ、じゅるっ、んん、んっ、ちゅっ、ちゅるっ、　んん、ちゅるっ、じゅるるるっ、じゅるっ、　ん、ん、んっ、んんッ！"

def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

stn_tst = get_text(japanese_phrase_cleaners(longtext), hps)
x = stn_tst.unsqueeze(0)
x_lengths = torch.LongTensor([stn_tst.size(0)])

torch.onnx.export(net_g, (x, x_lengths), "net_g.onnx", export_params=True, opset_version=16,
    do_constant_folding=True, input_names = ['modelInput'], output_names = ['modelOutput'],
    dynamic_axes={'modelInput' : {0 : 'batch_size'}, 'modelOutput' : {0 : 'batch_size'}}) 